In [2]:
import pyupbit
import time
from collections import deque
import datetime

# 잔고 조회 krw
def get_balance_krw():    
    balance = upbit.get_balance("KRW")
    return balance

# 잔고 조회 coin
def get_balance_wallet(ticker):
    balances = upbit.get_balances()
    b = True
    while (b):
        for i in range(len(balances)):
            a = ticker[4:7]
            if balances[i]['currency'] == a:
                balance = balances[i]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                c = (float(avg_buy_price), float(balance))
                b = False
            else:
                c = (int(0), int(0))
                b = False
    return c

def get_ticker_ma(ticker, df, volume):
    
    ma5 = df['close'].rolling(window=5).mean()
    df.insert(len(df.columns), "MA5", ma5)

    ma20 = df['close'].rolling(window=20).mean()
    df.insert(len(df.columns), "MA20", ma20)

    ma60 = df['close'].rolling(window=60).mean()
    df.insert(len(df.columns), "MA60", ma60)

    now_price = pyupbit.get_current_price(ticker)       # 코인의 현재가
    coin_check = get_balance_wallet(ticker) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0]   # 매수 평균가
    balance = coin_check[1]         # 코인 보유 개수

    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        if df['MA5'][-1] > df['MA20'][-1]:
            # 8천원치 매수
            buy_price = volume*0.95
            buy_order(ticker, buy_price)
        else:
            print('시세 감시 중')
            pass
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)

        if df['MA5'][-1] < df['MA20'][-1] or avg_price > now_price:
            sell_order(ticker, balance)
        else:
            print(f"코인명: {ticker}, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))

# 매수 주문
def buy_order(ticker, buy_price):
    try:
        while True:
            buy_result = upbit.buy_market_order(ticker, buy_price)
            if buy_result == None or 'error' in buy_result:
                print("매수 재 주문")
                time.sleep(1)
            else:
                print(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M','분', '%S','초'))
                print('매수 체결')
                return buy_result
    except:
        print("매수 주문 이상")

# 매도 주문
def sell_order(ticker, balance):
    try:
        while True:
            sell_result = upbit.sell_market_order(ticker, balance)
            if sell_result == None or 'error' in sell_result:
                print(f"{sell_result}, 매도 재 주문")
                time.sleep(1)
            else:
                print(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))
                print('매도 체결')
                return sell_result
    except:
        print("매도 주문 이상")

tickers = []
ma5 = []
ma20 = []
ma60 = []
# 원화로 거래 가능한 코인 리스트
tickers = pyupbit.get_tickers(fiat = 'KRW')
# ['KRW-BTC', 'KRW-ETH', 'KRW-NEO', 'KRW-MTL', 'KRW-LTC'] ... 등등
#살 가격 정하기
volume = 9000
# 4h봉 기준
interval = 'minute240'
# 로그인 정보 저장
upbit = pyupbit.Upbit('7dtWPaskQ2C2SGSa240M76QylZZEJvx5IjLSP1Vx', 'J57lzm4ppRwOtlhFo3Och3DtxvEI8PvbKxly9xj7')
# 종목 선택
ticker_select = tickers[1]
while True:
    df = pyupbit.get_ohlcv(ticker_select, interval = interval) # 데이터 프레임 생성
    get_ticker_ma(ticker_select, df, volume)
    # 시간 선택 지금은 30초 간격으로 거래 진행
    time.sleep(30)


시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중


KeyboardInterrupt: 